# AODSOC basic UART readout example
This example notebooks show how a user can read out the data from the AODSOC.
It is a basic examples

### Naludaq Version
*Max Version*: `0.17.2`  
*Min Version*: `0.17.2`

In [ ]:
# Print Naludaq version
import naludaq
print(f"Naludaq version: {naludaq.__version__}")

### Compatible Boards
+ `AARDVARCv3`
+ `HDSOCv1_evalr2`
+ `ASOCv3`
+ `AODSv2`
+ `TRBHM`
+ `AODSOC_AODS`
+ `AODSOC_ASOC`
+ `UPAC32`


## imports and variables

In [33]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
import time
from collections import deque
import numpy as np

# Imports for board creation and identification
from naludaq.board import Board, startup_board
from naludaq.tools.ftdi import list_ftdi_devices

# Registers modules handles communication with FPGA and ASIC registers.
from naludaq.communication import ControlRegisters, DigitalRegisters, AnalogRegisters

# Controllers controls one aspect of the board, board controllers start/stop acquisitons, readout controllers set the readout parameters.
from naludaq.controllers import get_board_controller, get_readout_controller

# Imports for data acquisition.
from naludaq.daq.workers.worker_serial_reader import SerialReader
from naludaq.daq.workers.packager import get_packager
import pickle

In [4]:
from logging import getLogger, Formatter, StreamHandler, INFO, DEBUG

def setup_logger(level=INFO):
    """Setup a basic logger.
    
    Logging to the stream formatted for easy visual readout.
    
    Args:
        level: Logging level, ex. logging.INFO
        
    Returns:
        logger object.
    """
    logger = getLogger()
    handler = StreamHandler()
    handler.setFormatter(Formatter('%(asctime)s %(name)-30s [%(levelname)-6s]: %(message)s'))
    logger.addHandler(handler)
    logger.setLevel(DEBUG)
    uart = getLogger('naludaq.board.connections._UART')
    uart.setLevel(DEBUG)
    
    return logger

try:
    logger.debug("logger already setup")
except:
    logger = setup_logger(DEBUG)

## Create board object

Use the function below to find the serialnumber of the board you want to connect to.

In [ ]:
list_ftdi_devices()

In [35]:
model, ser_no, baud = 'aodsoc_aods', 'B308B', 2_000_000 # AODS version
# model, ser_no = 'aodsoc_asoc', '5D4BB', 2_000_000 # ASOC version

In [36]:
BOARD = Board(model)
BOARD.load_registers()
BOARD.load_clockfile()

In [ ]:
BOARD.get_ftdi_connection(serial_number=ser_no, baud=baud)

In [ ]:
with BOARD:
    startup_board(BOARD)

# Readout unprocessed bytes

In [42]:
def readout_buffer(windows=2, lookback=2, amount=2, testmode=True) -> bytearray:
    """Readout data by filling the buffer and then reading it.
    
    This means reading out more data that the serailbuffer can hold will be discarded.
    
    Args:
        windows (int): Number of windows to readout.
        lookback (int): Number of windows to lookback.
        amount (int): Number of events to readout.
        testmode (bool): If True, readout will readout ASIC test pattern.

    Returns:
        bytearray: Data readout.
    """
    if amount > 10:
        print("amount too large, setting amount to 10")
        amount = 10
    _rc = get_readout_controller(BOARD)
    _bc = get_board_controller(BOARD)
    with BOARD:
        ControlRegisters(BOARD).write('testmode', False)
        DigitalRegisters(BOARD).write('enabletestpatt', testmode)
        while BOARD.connection.in_waiting:
            BOARD.connection.reset_input_buffer()
        _rc.number_events_to_read(amount)
        _rc.set_readout_channels(list(range(4)))
        _rc.set_read_window(windows=windows, lookback=lookback, write_after_trig=16)

        _bc.start_readout(trig='imm', lb='forced', readoutEn=True, singleEv=True)

        time.sleep(1*amount)

        _bc.stop_readout()

        print(f"Bytes in buffer: {BOARD.connection.in_waiting}")
        _data = BOARD.connection.read_all()
    return _data

In [ ]:
_data = readout_buffer(windows=2, lookback=2, amount=2, testmode=True)

## Readout uart with slicer
This will continuously read out the uart, store it in a buffer then have a separate thread slice the data when encountering `stopword`.

In [ ]:
def readout_packages(windows=2, lookback=2, amount=2, testmode=True) -> deque:
    """Readout data by usinga double buffer system.
    
    This allows for more data than the comport buffer can hold to be read.
    Comes at the cost of another layer of abstraction.
    
    Uses two worker threads, one that polls the virtual comport and one that slices the data into packages.
    
    Args:
        windows (int): Number of windows to readout.
        lookback (int): Number of windows to lookback.
        amount (int): Number of events to readout.
        testmode (bool): If True, readout will readout ASIC test pattern.
        
    Returns:
        deque: Data readout.
    """
    sbuf = deque()
    obuf = deque()
    stopword = BOARD.params['stop_word']
    sr = SerialReader(BOARD.connection, sbuf)
    pk = get_packager(BOARD, sbuf, obuf, deque(), stopword, 100)

    with BOARD:
        while BOARD.connection.in_waiting:
            BOARD.connection.reset_input_buffer()
        DigitalRegisters(BOARD).write('enabletestpatt', testmode)
        _rc = get_readout_controller(BOARD)
        _bc = get_board_controller(BOARD)

        _rc.number_events_to_read(amount)
        _rc.set_readout_channels(list(range(4)))
        _rc.set_read_window(windows=windows, lookback=lookback, write_after_trig=16)

        _bc.start_readout(trig='imm', lb='forced', singleEv=True)
        pk.start()
        sr.start()
        time.sleep(1*amount)
        sr.stop()
        pk.stop()
        _bc.stop_readout()
    return obuf

In [ ]:
_pkdata = readout_packages(windows=2, lookback=2, amount=2, testmode=True)

In [ ]:
with open('data.dat', 'wb') as f:
    pickle.dump(_pkdata, f)

# Print the data
Semple printout to show the data, this is a function to visualize the data and is highly dependent on the board `model`.
The current sample is for the `AODSOC` boards.

In [ ]:
_words = np.frombuffer(_data, '>H')

samples = 64
num_window_headers = 1
num_channels = 4
channel_footers = 0
num_evt_headers = 3
channels_per_chip = 4

step_size = (samples + num_window_headers) * num_channels + channel_footers
header_steps = range(0, len(_words)-step_size, step_size+num_evt_headers+1)
channel_step_size = (samples + num_window_headers)

windmask = 0b0000_0000_1111_1111
chanmask = 0b0000_0011_0000_0000
chanshift = 8

err = 0
for idx in header_steps:
    window_steps = range(idx+num_evt_headers, idx+step_size-1, samples+num_window_headers)
    chipnum = _words[idx] >> 12
    
    
    print(' Next Chip '.center(80, '#'))
    print('Chip number:', chipnum)
    print('Headers:', _words[idx:idx+num_evt_headers])

    for inner_idx in window_steps:
        # First two words are window and channel
        window = _words[inner_idx] & windmask
        channel = (_words[inner_idx] & chanmask >> chanshift) #  + (chipnum*channels_per_chip)
        data = _words[inner_idx+num_window_headers:inner_idx+channel_step_size]
        
        print(' Next Data '.center(80, '-'))
        print('Window Header:', _words[inner_idx], bin(_words[inner_idx]))
        print('Window:', window, 'Channel:', channel)
        print('Data len:', len(data))
        print('Data:', [f'{x:04x}' for x in data])
        
